In [2]:
from bs4 import BeautifulSoup
import requests
import os
from PIL import Image
from io import BytesIO

# Specify the image categories and their respective URLs
categories = {
    'canal': ['https://www.google.com/search?q=canal+problem+in+bangkok&tbm=isch&ved=2ahUKEwiQ-cfEuff-AhXk_TgGHfK8D84Q2-cCegQIABAA&oq=canal+problem+in+bangkok&gs_lcp=CgNpbWcQAzoECCMQJ1CABVjSHWCJIGgBcAB4AIABX4gB5AaSAQIxMJgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=RDZiZJDHK-T74-EP8vm-8Aw&bih=802&biw=1707', 
              'https://www.google.com/search?q=water+hyacinth+canal+in+thailand&tbm=isch&ved=2ahUKEwjt_Zrxuff-AhWA1nMBHXU2Dq8Q2-cCegQIABAA&oq=water+hyacinth+canal+in+thailand&gs_lcp=CgNpbWcQA1AAWLKqAmCrrAJoAXAAeACAAY0BiAHLCpIBBDE0LjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=ojZiZO2kFYCtz7sP9ey4-Ao&bih=802&biw=1707'],
    'electric': ['https://www.google.com/search?q=electric+wire+in+thailand&tbm=isch&ved=2ahUKEwjxmMObuvf-AhW4k9gFHQAUBJQQ2-cCegQIABAA&oq=electric+wire+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJzoFCAAQgAQ6CAgAEIAEELEDOggIABCxAxCDAToHCAAQigUQQzoECAAQAzoKCAAQigUQsQMQQzoECAAQHjoHCAAQExCABDoICAAQCBAeEBNQ1ypYtGFghGNoAHAAeACAAYoBiAGwE5IBBDI1LjKYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=-zZiZLGvBbin4t4PgKiQoAk&bih=802&biw=1707',
                  'https://www.google.com/search?q=electric+problem+image+in+thailand&tbm=isch&ved=2ahUKEwio4Prtuvf-AhWej2MGHVKOCREQ2-cCegQIABAA&oq=electric+problem+image+in+thailand&gs_lcp=CgNpbWcQA1AAWIVBYIBDaAFwAHgAgAFpiAGhBpIBAzcuMpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=pzdiZOjwOp6fjuMP0pymiAE&bih=802&biw=1707'],
    'flooding': ['https://www.google.com/search?q=flooding+image+in+thailand&tbm=isch&ved=2ahUKEwjY2qH5uvf-AhWr9nMBHSwJBPAQ2-cCegQIABAA&oq=flooding+image+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1CcQ1iwTWD3TmgBcAB4AIABb4gBqgeSAQM4LjKYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=vzdiZJj_KKvtz7sPrJKQgA8&bih=802&biw=1707', 
                 'https://www.google.com/search?q=road+flooding+image+in+bangkok&tbm=isch&ved=2ahUKEwixuJbKu_f-AhVZxnMBHWSgBKwQ2-cCegQIABAA&oq=road+flooding+image+in+bangkok&gs_lcp=CgNpbWcQAzoECCMQJ1DtHlj0LmD4MGgAcAB4AIABXIgBnwWSAQE4mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=aThiZPHfFdmMz7sP5MCS4Ao&bih=802&biw=1707'],
    'light': ['https://www.google.com/search?q=light+in+thailand+road&tbm=isch&ved=2ahUKEwjv38nGvff-AhVW_jgGHcw2BLkQ2-cCegQIABAA&oq=light+in+thailand+road&gs_lcp=CgNpbWcQAzoECCMQJ1D3CFiEEWDnE2gAcAB4AIABY4gB6wKSAQE0mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=ejpiZO-6Kdb84-EPzO2QyAs&bih=802&biw=1707',
               'https://www.google.com/search?q=dark+light+in+thailand+road&tbm=isch&ved=2ahUKEwje2fLMvff-AhXu6nMBHYVKCpYQ2-cCegQIABAA&oq=dark+light+in+thailand+road&gs_lcp=CgNpbWcQAzoECCMQJ1CmfljahgFglogBaABwAHgAgAFyiAHABJIBAzUuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=hzpiZJ76OO7Vz7sPhZWpsAk&bih=802&biw=1707'],
    'road': ['https://www.google.com/search?q=road+crack+in+thailand&tbm=isch&ved=2ahUKEwjHgYOhvvf-AhWBpOkKHVrcCNgQ2-cCegQIABAA&oq=road+crack+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJzoFCAAQgAQ6BggAEAcQHlDdDFjizwFg2dEBaAJwAHgAgAFliAHBBpIBAzkuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=ODtiZMeFFoHJpgfauKPADQ&bih=802&biw=1707', 
             'https://www.google.com/search?q=road+pothole+in+thailand&tbm=isch&ved=2ahUKEwiF5Y6kv_f-AhX8zqACHTMqAsEQ2-cCegQIABAA&oq=road+pothole+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJzoFCAAQgAQ6BggAEAcQHlD_Clj4HWD4I2gBcAB4AIABc4gBjgeSAQQxMC4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=SzxiZMWTEfydg8UPs9SIiAw&bih=802&biw=1707'],
    'sanitary': ['https://www.google.com/search?q=gabage+on+road+image+in+thailand&tbm=isch&ved=2ahUKEwi-yaGrwPf-AhWQgGMGHYHnA8QQ2-cCegQIABAA&oq=gabage+on+road+image+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1DBDljXG2D2HGgAcAB4AIABaogB6AWSAQM4LjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=Zj1iZL7-KpCBjuMPgc-PoAw&bih=802&biw=1707', 
                 'https://www.google.com/search?q=gabage+in+river+image+in+thailand&tbm=isch&ved=2ahUKEwji0-ulwPf-AhVkSnwKHUE1C5IQ2-cCegQIABAA&oq=gabage+in+river+image+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1CjCFjRFmDTIWgAcAB4AIABWYgBzwSSAQE3mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=Wz1iZKK6FOSU8QPB6qyQCQ&bih=802&biw=1707'],
    'sewer': ['https://www.google.com/search?q=sewer++on+road+image+in+thailand&tbm=isch&ved=2ahUKEwig4LHrwPf-AhVh_DgGHcxZDSQQ2-cCegQIABAA&oq=sewer++on+road+image+in+thailand&gs_lcp=CgNpbWcQAzIECCMQJ1DqBli8EmCTFGgAcAB4AIABnQGIAYoGkgEDOC4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=7T1iZOC1C-H44-EPzLO1oAI&bih=802&biw=1707', 
              'https://www.google.com/search?q=sewer+on+sidewalk+image+in+thailand&tbm=isch&ved=2ahUKEwjM6ruewff-AhWGEbcAHWrtCmcQ2-cCegQIABAA&oq=sewer+on+sidewalk+image+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1D1CFjAKGD6L2gBcAB4AIABqgGIAYEKkgEDOS40mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=WD5iZMzeEoaj3LUP6tqruAY&bih=802&biw=1707'],
    'sidewalk': ['https://www.google.com/search?q=sidewalk+image+in+thailand&tbm=isch&ved=2ahUKEwiJnYipwff-AhVrxXMBHQo7CuEQ2-cCegQIABAA&oq=sidewalk+image+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1CLBliLBmC8CmgBcAB4AIABWYgBWZIBATGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=bj5iZImuIOuKz7sPivaoiA4&bih=802&biw=1707', 
                 'https://www.google.com/search?q=pavement+image+in+bangkok&tbm=isch&ved=2ahUKEwjFr_KSwvf-AhWuzaACHXHxC7AQ2-cCegQIABAA&oq=pavement+image+in+bangkok&gs_lcp=CgNpbWcQAzoECCMQJzoGCAAQBxAeOggIABAFEAcQHlDuBliAJmCbKGgAcAB4AIABXYgB-QmSAQIxNZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=TD9iZMXZHK6bg8UP8eKvgAs&bih=802&biw=1707'],
    'stray': ['https://www.google.com/search?q=stray+image+in+thailand&tbm=isch&ved=2ahUKEwje7du2wvf-AhUqjtgFHZRNA2sQ2-cCegQIABAA&oq=stray+image+in+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1DTDVjTDWCnEGgAcAB4AIABWYgBqgGSAQEymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=lz9iZJ7GJKqc4t4PlJuN2AY&bih=802&biw=1707', 
              'https://www.google.com/search?q=stray+on+roadin+thailand&tbm=isch&ved=2ahUKEwiIo-TZwvf-AhX9zqACHXy6BVsQ2-cCegQIABAA&oq=stray+on+roadin+thailand&gs_lcp=CgNpbWcQAzoECCMQJ1DDCljDCmCoDGgAcAB4AIABW4gBqQGSAQEymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=4T9iZMiuCP2dg8UP_PSW2AU&bih=802&biw=1707'],
    'traffic': ['https://www.google.com/search?q=trAffic+images+in+thailand&sxsrf=APwXEddAu71jTC91TyrcNrDmsnhWMjiR9w:1683883796657&source=lnms&tbm=isch&sa=X&ved=2ahUKEwihpKv8u-_-AhU6a2wGHUDPCIcQ_AUoAXoECAEQAw&biw=1920&bih=909&dpr=1',
                 'https://www.google.com/search?q=traffic+images+in+thailand&tbm=isch&chips=q:traffic+thailand,online_chips:congestion:8sY58KcDMng%3D&hl=en-GB&sa=X&ved=2ahUKEwict6rJwO_-AhV6ALcAHYV_DRQQ4lYoCnoECAEQOQ&biw=1905&bih=909']
}

# Create a folder for each category
# for category in categories:
#     os.makedirs(category, exist_ok=True)

# Scrape images for each category and URL
for category, urls in categories.items():
    # Get the last image number for the category
    next_image_number = 0

    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        image_tags = soup.find_all('img')
        for i, image_tag in enumerate(image_tags):
            image_url = image_tag['src']
            if image_url.startswith('https'):
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content))
                img.save(f"train/train_scraped_{category}_img_{next_image_number + i}.jpg", "JPEG")
        next_image_number += len(image_tags)
        
print("Finished")

Finished
